In [1]:
import tensorflow as tf
print(tf.__version__)

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D, Dense, Dropout, LayerNormalization, Bidirectional, LSTM, GRU, Layer, SpatialDropout1D, GlobalAveragePooling1D
from tensorflow.keras.layers import Lambda, Reshape, Flatten, Input, MultiHeadAttention, Flatten, Concatenate, Add, Multiply, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

2025-12-19 15:49:35.210009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.20.0


In [2]:
import torch
import pandas as pd
import os, math, random
import numpy as np
import pyrsgis
from pyrsgis import raster
from osgeo import gdal
from tensorflow.keras.models import load_model
from scipy import interpolate
import rasterio
import gc

In [3]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
class MultiFocalLossSlopeElevationConstraint(tf.keras.losses.Loss):
    def __init__(self, slope_threshold=40, elevation_threshold=2000,
                 alpha=0.25, gamma=2.0,
                 lambda_slope=0.4, lambda_elevation=0.3,
                 name='multi_focal_loss_slope_elevation_constraint'):
        super().__init__(name=name)
        self.slope_threshold = slope_threshold
        self.elevation_threshold = elevation_threshold
        self.alpha = alpha
        self.gamma = gamma
        self.lambda_slope = lambda_slope
        self.lambda_elevation = lambda_elevation
    
    def call(self, y_true, y_pred):
        return tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'slope_threshold': self.slope_threshold,
            'elevation_threshold': self.elevation_threshold,
            'alpha': self.alpha,
            'gamma': self.gamma,
            'lambda_slope': self.lambda_slope,
            'lambda_elevation': self.lambda_elevation
        })
        return config

In [5]:
class ALPELayer(Layer):
    """
    Attention-based Learnable Positional Encoding as a custom Keras Layer
    """
    def __init__(self, n_timesteps=24, embed_dim=64, dropout_rate=0.5, name='alpe', **kwargs):
        super(ALPELayer, self).__init__(name=name, **kwargs)
        self.n_timesteps = n_timesteps
        self.embed_dim = embed_dim
        self.dropout_rate = dropout_rate
        
    def build(self, input_shape):
        """
        Build layer components
        input_shape: tuple of (x_shape, mask_shape)
        """
        feature_dim = input_shape[0][-1]  # Get feature dimension from x
        
        # Positional embedding layer
        self.pos_embedding = Embedding(
            input_dim=self.n_timesteps,
            output_dim=self.embed_dim,
            name=f'{self.name}_pos_embed')
        
        # 1D Convolution for learning interpolation patterns
        self.conv1d = Conv1D(
            filters=self.embed_dim,
            kernel_size=3,
            padding='same',
            activation='relu',
            name=f'{self.name}_conv1d')
        
        # ECA: Adaptive kernel size
        k_size = max(3, int(abs((np.log2(self.embed_dim) + 1) / 2)))
        if k_size % 2 == 0:
            k_size += 1
        
        # ECA channel attention
        self.eca_conv = Conv1D(
            filters=1,
            kernel_size=k_size,
            padding='same',
            activation='sigmoid',
            name=f'{self.name}_eca')
        
        # Projection to match input feature dimension
        if self.embed_dim != feature_dim:
            self.projection = Dense(
                units=feature_dim,
                name=f'{self.name}_projection')
        else:
            self.projection = None

        # Add dropout layer
        self.dropout = Dropout(self.dropout_rate, name=f'{self.name}_dropout')
        
        # Store feature_dim for rebuild
        self.feature_dim = feature_dim
        
        # CRITICAL: Actually build the sub-layers
        self.pos_embedding.build((None,))  # Embedding input shape
        self.conv1d.build((None, self.n_timesteps, self.embed_dim))
        
        k_size = max(3, int(abs((np.log2(self.embed_dim) + 1) / 2)))
        if k_size % 2 == 0:
            k_size += 1
        self.eca_conv.build((None, 1, self.embed_dim))
        
        if self.projection is not None:
            self.projection.build((None, self.n_timesteps, self.embed_dim))
        
        self.dropout.build((None, self.n_timesteps, feature_dim))
        
        super(ALPELayer, self).build(input_shape)
    
    def call(self, inputs, training=None):
        """
        Forward pass
        inputs: [x, confidence_mask]
            x: [batch, timesteps, features]
            confidence_mask: [batch, timesteps]
        """
        x, confidence_mask = inputs
        
        # ============================================================
        # STEP 1: Create positional embeddings
        # ============================================================
        position_ids = tf.range(self.n_timesteps)
        pos_embedding = self.pos_embedding(position_ids)  # [timesteps, embed_dim]
        
        # Broadcast to batch dimension
        batch_size = tf.shape(x)[0]
        pos_embedding = tf.tile(
            tf.expand_dims(pos_embedding, axis=0),
            [batch_size, 1, 1]
        )  # [batch, timesteps, embed_dim]
        
        # Apply conf mask
        mask_expanded = tf.expand_dims(confidence_mask, axis=-1)  # [batch, timesteps, 1]
        mask_expanded = tf.cast(mask_expanded, tf.float32)
        
        # Invert mask: confidence=1.0 → weight=0.0, confidence=0.0 → weight=1.0
        alpe_weight = 1.0 - mask_expanded  # [batch, timesteps, 1]
        
        # Apply weight to positional encoding
        masked_pe = pos_embedding * alpe_weight  # [batch, timesteps, embed_dim]

        # Keras Conv1D expects [batch, timesteps, channels]
        # masked_pe is already in correct shape: [batch, timesteps, embed_dim]
        pe_conv = self.conv1d(masked_pe)  # [batch, timesteps, embed_dim]
        
        # For ECA on channels, we need to work on channel dimension
        # Transpose to [batch, embed_dim, timesteps] for channel-wise operations
        pe_transposed = tf.transpose(pe_conv, perm=[0, 2, 1])  # [batch, embed_dim, timesteps]
        
        # Global average pooling across time
        gap = tf.reduce_mean(pe_transposed, axis=-1, keepdims=True)  # [batch, embed_dim, 1]
        
        # Reshape for Conv1D: [batch, embed_dim, 1] → need [batch, timesteps=embed_dim, channels=1]
        gap_reshaped = tf.transpose(gap, perm=[0, 2, 1])  # [batch, 1, embed_dim]
        
        # Channel attention via 1D conv
        channel_att = self.eca_conv(gap_reshaped)  # [batch, 1, embed_dim]
        
        # Reshape back: [batch, 1, embed_dim] → [batch, embed_dim, 1]
        channel_att = tf.transpose(channel_att, perm=[0, 2, 1])  # [batch, embed_dim, 1]
        
        # Apply attention to pe_transposed [batch, embed_dim, timesteps]
        pe_attended = pe_transposed * channel_att  # [batch, embed_dim, timesteps]
        
        # ============================================================
        # STEP 5: Transpose back to [batch, timesteps, embed_dim]
        # ============================================================
        alpe_output = tf.transpose(pe_attended, perm=[0, 2, 1])  # [batch, timesteps, embed_dim]
        
        # Project to match input feature dimension if needed
        if self.projection is not None:
            alpe_output = self.projection(alpe_output)  # [batch, timesteps, features]

        alpe_output = self.dropout(alpe_output, training=training)
        
        # ============================================================
        # STEP 6: Residual connection with weighting
        # ============================================================
        # Now shapes match:
        # alpe_output: [batch, timesteps, features]
        # alpe_weight: [batch, timesteps, 1]
        alpe_contribution = alpe_output * alpe_weight  # Broadcasting works!
        x_enhanced = x + alpe_contribution
        
        return x_enhanced
    
    def compute_output_shape(self, input_shape):
        return input_shape[0]  # Return x's shape
    
    def get_config(self):
        config = super(ALPELayer, self).get_config()
        config.update({
            'n_timesteps': self.n_timesteps,
            'embed_dim': self.embed_dim,
            'dropout_rate': self.dropout_rate
        })
        return config
    
    def get_build_config(self):
        """Return configuration needed to rebuild the layer during loading"""
        build_config = {
            'feature_dim': getattr(self, 'feature_dim', None)
        }
        return build_config
    
    def build_from_config(self, config):
        """Rebuild the layer's internal state from config"""
        feature_dim = config.get('feature_dim')
        if feature_dim is not None:
            # Reconstruct input_shape that build() expects
            input_shape = (
                (None, self.n_timesteps, feature_dim),  # x_shape
                (None, self.n_timesteps)  # mask_shape
            )
            self.build(input_shape)

In [6]:
model_path = '/home/jupyter-bryan/ISA_Data/ISA_Citarum_Multi_Orig_withALPE.keras'
model = load_model(model_path,
                   custom_objects={'ALPELayer': ALPELayer,
                                   'MultiFocalLossSlopeElevationConstraint': MultiFocalLossSlopeElevationConstraint},
                   compile=False)

# Then recompile with your custom loss
# model.compile(
#     optimizer=Adam(learning_rate=0.001),
#     loss=MultiFocalLossSlopeElevationConstraint(),
#     metrics=['accuracy'])

I0000 00:00:1766130579.597022  474247 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1088 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [7]:
# model = load_model('/home/jupyter-bryan/ISA_Data/ISA_Citarum_Multi_Orig_CCEL.h5')

In [9]:
# # Load the saved model
# model_path = '/home/jupyter-bryan/ISA_Data/ISA_Citarum_Multi_Orig.keras'

# model = load_model(model_path,
#                    custom_objects={'MultiFocalLossSlopeElevationConstraint': MultiFocalLossSlopeElevationConstraint},
#                    compile=False)

# Load a new multispectral image
ds_ndvi, img_ndvi  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/DKI_3-Month_NDVI_2019_2024.tif')
ds_mndwi, img_mndwi  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/DKI_3-Month_MNDWI_2019_2024.tif')
ds_ndbi, img_ndbi  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/DKI_3-Month_NDBI_2019_2024.tif')
ds_ndbsi, img_ndbsi  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/DKI_3-Month_NDBSI_2019_2024.tif')
ds_cbi, img_cbi  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/DKI_3-Month_CBI_2019_2024.tif')
ds_uci, img_uci  = raster.read('/home/jupyter-bryan/ISA_Data/Raster/DKI_3-Month_UCI_2019_2024.tif')

print(img_ndvi.shape)
print(img_mndwi.shape)
print(img_ndbi.shape)
print(img_ndbsi.shape)
print(img_cbi.shape)
print(img_uci.shape)

Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed


(24, 3147, 3194)
(24, 3147, 3194)
(24, 3147, 3194)
(24, 3147, 3194)
(24, 3147, 3194)
(24, 3147, 3194)


Warning 1: PROJ: proj_create_from_database: Open of /opt/tljh/user/share/proj failed


In [10]:
img_ndvi_reshape = img_ndvi.reshape(img_ndvi.shape[0],(img_ndvi.shape[1]*img_ndvi.shape[2])).T
img_mndwi_reshape = img_mndwi.reshape(img_mndwi.shape[0],(img_mndwi.shape[1]*img_mndwi.shape[2])).T
img_ndbi_reshape = img_ndbi.reshape(img_ndbi.shape[0],(img_ndbi.shape[1]*img_ndbi.shape[2])).T
img_ndbsi_reshape = img_ndbsi.reshape(img_ndbsi.shape[0],(img_ndbsi.shape[1]*img_ndbsi.shape[2])).T
img_cbi_reshape = img_cbi.reshape(img_cbi.shape[0],(img_cbi.shape[1]*img_cbi.shape[2])).T
img_uci_reshape = img_uci.reshape(img_uci.shape[0],(img_uci.shape[1]*img_uci.shape[2])).T

In [11]:
def clean_and_interpolate_with_mask(data):
    """
    Interpolate missing data AND create confidence mask
    Returns: (interpolated_data, confidence_mask)
    """
    df = pd.DataFrame(data, dtype=np.float32)
    
    # Track original missing values BEFORE interpolation
    original_missing_mask = df.isnull()  # True = missing
    
    if len(df) > 100000:
        print(f"Processing large dataset with {len(df):,} pixels in chunks...")
        chunk_size = 50000
        results = []
        masks = []
        
        for i in range(0, len(df), chunk_size):
            chunk = df.iloc[i:i+chunk_size].copy()
            chunk_missing = original_missing_mask.iloc[i:i+chunk_size].copy()
            
            # Interpolate
            chunk_linear = chunk.interpolate(method='linear', limit_direction='both', axis=1, limit=None)
            chunk_spline = chunk_linear.interpolate(method='spline', order=3, axis=1)
            chunk_filled = chunk_spline.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
            
            # Identify what STILL missing after interpolation
            still_missing = chunk_filled.isnull()
            
            # Fill remaining with zeros
            chunk_final = chunk_filled.fillna(0)
            chunk_clipped = chunk_final.clip(-1e6, 1e6)
            
            # Create confidence mask
            # 1.0 = valid or successfully interpolated
            # 0.0 = zero-filled (ALPE will handle)
            chunk_mask = np.ones_like(chunk_clipped.values, dtype=np.float32)
            chunk_mask[still_missing.values] = 0.0
            
            results.append(chunk_clipped.values.astype(np.float32))
            masks.append(chunk_mask)
            
            print(f"Processed chunk {i//chunk_size + 1}/{(len(df)-1)//chunk_size + 1}")
            gc.collect()
        
        valid_result = np.vstack(results)
        confidence_mask = np.vstack(masks)
    else:
        # Small dataset processing
        df_linear = df.interpolate(method='linear', limit_direction='both', axis=1, limit=None)
        df_spline = df_linear.interpolate(method='spline', order=3, axis=1)
        df_filled = df_spline.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
        
        still_missing = df_filled.isnull()
        df_final = df_filled.fillna(0)
        valid_result = df_final.clip(-1e6, 1e6).values.astype(np.float32)
        
        # Create mask
        confidence_mask = np.ones_like(valid_result, dtype=np.float32)
        confidence_mask[still_missing.values] = 0.0
    
    print(f"  Pixels with zero-filled data: {(confidence_mask == 0.0).any(axis=1).sum()}")
    
    return valid_result, confidence_mask

In [12]:
# def clean_and_interpolate(data):
#     df = pd.DataFrame(data, dtype=np.float32) 
#     df.dropna(how='all')
    
#     if len(df) > 100000:  
#         print(f"Processing large dataset with {len(df):,} pixels in chunks...")
#         chunk_size = 50000
#         results = []
#         for i in range(0, len(df), chunk_size):
#             chunk = df.iloc[i:i+chunk_size].copy()
#             chunk_linear = chunk.interpolate(method='linear', limit_direction='both', axis=1, limit=None)
#             chunk_final = chunk_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
#             chunk_clipped = chunk_final.clip(-1e6, 1e6)
#             results.append(chunk_clipped.values.astype(np.float32))
#             print(f"Processed chunk {i//chunk_size + 1}/{(len(df)-1)//chunk_size + 1}")
#             gc.collect()
#         valid_result = np.vstack(results)
#     else:
#         df_linear = df.interpolate(method='linear', limit_direction='both', axis=1, limit=None)
#         df_final = df_linear.interpolate(method='spline', order=3, axis=1).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)
#         valid_result = df_final.clip(-1e6, 1e6).values.astype(np.float32)

#     return valid_result

In [13]:
# %%time
# print("\n=== Interpolating NDVI ===")
# img_ndvi_int = clean_and_interpolate(img_ndvi_reshape)

# print("\n=== Interpolating MNDWI ===")
# img_mndwi_int = clean_and_interpolate(img_mndwi_reshape)

# print("\n=== Interpolating NDBI ===")
# img_ndbi_int = clean_and_interpolate(img_ndbi_reshape)

# print("\n=== Interpolating NDBSI ===")
# img_ndbsi_int = clean_and_interpolate(img_ndbsi_reshape)

# print("\n=== Interpolating CBI ===")
# img_cbi_int = clean_and_interpolate(img_cbi_reshape)

# print("\n=== Interpolating UCI ===")
# img_uci_int = clean_and_interpolate(img_uci_reshape)

In [14]:
%%time
print("\n=== Interpolating NDVI ===")
img_ndvi_int, mask_ndvi = clean_and_interpolate_with_mask(img_ndvi_reshape)

print("\n=== Interpolating MNDWI ===")
img_mndwi_int, mask_mndwi = clean_and_interpolate_with_mask(img_mndwi_reshape)

print("\n=== Interpolating NDBI ===")
img_ndbi_int, mask_ndbi = clean_and_interpolate_with_mask(img_ndbi_reshape)

print("\n=== Interpolating NDBSI ===")
img_ndbsi_int, mask_ndbsi = clean_and_interpolate_with_mask(img_ndbsi_reshape)

print("\n=== Interpolating CBI ===")
img_cbi_int, mask_cbi = clean_and_interpolate_with_mask(img_cbi_reshape)

print("\n=== Interpolating UCI ===")
img_uci_int, mask_uci = clean_and_interpolate_with_mask(img_uci_reshape)


=== Interpolating NDVI ===
Processing large dataset with 10,051,518 pixels in chunks...
Processed chunk 1/202
Processed chunk 2/202
Processed chunk 3/202
Processed chunk 4/202
Processed chunk 5/202
Processed chunk 6/202
Processed chunk 7/202
Processed chunk 8/202
Processed chunk 9/202
Processed chunk 10/202
Processed chunk 11/202
Processed chunk 12/202
Processed chunk 13/202
Processed chunk 14/202
Processed chunk 15/202
Processed chunk 16/202
Processed chunk 17/202
Processed chunk 18/202
Processed chunk 19/202
Processed chunk 20/202
Processed chunk 21/202
Processed chunk 22/202
Processed chunk 23/202
Processed chunk 24/202
Processed chunk 25/202
Processed chunk 26/202
Processed chunk 27/202
Processed chunk 28/202
Processed chunk 29/202
Processed chunk 30/202
Processed chunk 31/202
Processed chunk 32/202
Processed chunk 33/202
Processed chunk 34/202
Processed chunk 35/202
Processed chunk 36/202
Processed chunk 37/202
Processed chunk 38/202
Processed chunk 39/202
Processed chunk 40/202


In [15]:
comb_all = np.stack([img_ndvi_int, img_mndwi_int, img_ndbi_int, 
                     img_ndbsi_int, img_cbi_int, img_uci_int], axis=-1)

print(f"Combined data shape: {comb_all.shape}")  # Should be [n_pixels, 24, 6]

# Stack the MASKS
confidence_mask_combined = np.stack([mask_ndvi, mask_mndwi, mask_ndbi,
                                     mask_ndbsi, mask_cbi, mask_uci], axis=-1)  # [n_pixels, 24, 6]

confidence_mask_final = confidence_mask_combined.mean(axis=-1)
print(f"Final mask shape: {confidence_mask_combined.shape}")
print(f"Final mask shape: {confidence_mask_final.shape}") 

# # Clean up
# del img_ndvi_int, img_mndwi_int, img_ndbi_int, img_ndbsi_int, img_cbi_int, img_uci_int
# del img_ndvi_reshape, img_mndwi_reshape, img_ndbi_reshape, img_ndbsi_reshape, img_cbi_reshape, img_uci_reshape
# del mask_ndvi, mask_mndwi, mask_ndbi, mask_ndbsi, mask_cbi, mask_uci
# # del confidence_mask_combined
# gc.collect()

Combined data shape: (10051518, 24, 6)
Final mask shape: (10051518, 24, 6)
Final mask shape: (10051518, 24)


In [16]:
# # Stack all bands
# print("Stacking all bands...")
# comb_all = np.stack((img_ndvi_int, img_mndwi_int, 
#                      img_ndbi_int, img_ndbsi_int, img_cbi_int, img_uci_int), axis=1)

# # Reshape for model input
# comb_all_reshape = comb_all.reshape(comb_all.shape[0], comb_all.shape[2], comb_all.shape[1])
# print(f"Final input shape: {comb_all_reshape.shape}")

# # Clear intermediate variables to free memory
# del img_ndvi_int, img_mndwi_int, img_ndbi_int, img_ndbsi_int, img_cbi_int, img_uci_int
# del img_ndvi_reshape, img_mndwi_reshape, img_ndbi_reshape, img_ndbsi_reshape, img_cbi_reshape, img_uci_reshape
# del comb_all
# gc.collect()

In [17]:
# comb_all = np.stack((img_ndvi_int, img_mndwi_int, 
#                      img_ndbi_int, img_ndbsi_int, 
#                      img_cbi_int, img_uci_int), axis=1)

# # Reshape: [n_pixels, 6_bands, n_timesteps] → [n_pixels, n_timesteps, 6_bands]
# comb_all_reshape = comb_all.reshape(comb_all.shape[0], comb_all.shape[2], comb_all.shape[1])
# print(f"Final input shape: {comb_all_reshape.shape}")  # Should be [n_pixels, 24, 6]
# print(f"Confidence mask shape: {confidence_mask_combined.shape}")  # Should be [n_pixels, 24]

# # Clear intermediate variables
# del img_ndvi_int, img_mndwi_int, img_ndbi_int, img_ndbsi_int, img_cbi_int, img_uci_int
# del img_ndvi_reshape, img_mndwi_reshape, img_ndbi_reshape, img_ndbsi_reshape, img_cbi_reshape, img_uci_reshape
# del mask_ndvi, mask_mndwi, mask_ndbi, mask_ndbsi, mask_cbi, mask_uci, masks_stacked
# del comb_all
# gc.collect()

In [18]:
# # BATCH PREDICTION - Process in smaller chunks to avoid memory issues
# def predict_in_batches(model, data, batch_size=5000):
#     """Predict data in batches to manage memory"""
#     n_samples = data.shape[0]
#     predictions = []
    
#     print(f"Predicting {n_samples:,} pixels in batches of {batch_size:,}...")
    
#     for i in range(0, n_samples, batch_size):
#         end_idx = min(i + batch_size, n_samples)
#         batch = data[i:end_idx]
#         batch = np.asarray(batch, dtype=np.float32)
        
#         try:
#             batch_pred = model.predict(batch, verbose=0)
#             predictions.append(batch_pred)
#             print(f"Processed batch {i//batch_size + 1}/{(n_samples-1)//batch_size + 1}")
#         except Exception as e:
#             print(f"Error in batch {i//batch_size + 1}: {e}")
#             # Try with smaller batch size
#             smaller_batch_size = batch_size // 2
#             print(f"Retrying with smaller batch size: {smaller_batch_size}")
#             for j in range(i, end_idx, smaller_batch_size):
#                 small_end = min(j + smaller_batch_size, end_idx)
#                 small_batch = data[j:small_end]
#                 small_batch = np.asarray(small_batch, dtype=np.float32)
#                 small_pred = model.predict(small_batch, verbose=0)
#                 predictions.append(small_pred)
        
#         tf.keras.backend.clear_session()
#         gc.collect()
    
#     return np.vstack(predictions)

In [19]:
def predict_in_batches_with_mask(model, data, masks, batch_size=5000):
    """
    Predict data in batches with confidence masks
    model: Keras model expecting [data, mask] inputs
    data: [n_pixels, n_timesteps, n_bands]
    masks: [n_pixels, n_timesteps]
    """
    n_samples = data.shape[0]
    predictions = []
    
    print(f"Predicting {n_samples:,} pixels in batches of {batch_size:,}...")
    
    for i in range(0, n_samples, batch_size):
        end_idx = min(i + batch_size, n_samples)
        batch_data = data[i:end_idx]
        batch_mask = masks[i:end_idx]
        
        # Ensure correct dtypes
        batch_data = np.asarray(batch_data, dtype=np.float32)
        batch_mask = np.asarray(batch_mask, dtype=np.float32)
        
        try:
            # CRITICAL: Pass BOTH inputs as a list
            batch_pred = model.predict([batch_data, batch_mask], verbose=0)
            predictions.append(batch_pred)
            
            if (i // batch_size + 1) % 10 == 0:  # Print every 10 batches
                print(f"Processed batch {i//batch_size + 1}/{(n_samples-1)//batch_size + 1}")
        
        except Exception as e:
            print(f"Error in batch {i//batch_size + 1}: {e}")
            # Try with smaller batch size
            smaller_batch_size = batch_size // 2
            print(f"Retrying with smaller batch size: {smaller_batch_size}")
            
            for j in range(i, end_idx, smaller_batch_size):
                small_end = min(j + smaller_batch_size, end_idx)
                small_batch_data = data[j:small_end]
                small_batch_mask = masks[j:small_end]
                
                small_batch_data = np.asarray(small_batch_data, dtype=np.float32)
                small_batch_mask = np.asarray(small_batch_mask, dtype=np.float32)
                
                small_pred = model.predict([small_batch_data, small_batch_mask], verbose=0)
                predictions.append(small_pred)
        
        # Clear session periodically to avoid memory buildup
        if (i // batch_size + 1) % 50 == 0:
            tf.keras.backend.clear_session()
            gc.collect()
    
    print("Combining predictions...")
    return np.vstack(predictions)

In [20]:
# %%time
# # Perform prediction
# print("Starting prediction...")
# try:
#     pred_ = predict_in_batches(model, comb_all_reshape, batch_size=2000)
    
# except Exception as e:
#     print(f"Error during prediction: {e}")
#     print("Trying with even smaller batch size...")
#     try:
#         pred_ = predict_in_batches(model, comb_all_reshape, batch_size=1000)
#     except Exception as e2:
#         print(f"Still getting error: {e2}")
#         print("Trying CPU-only prediction...")
#         with tf.device('/CPU:0'):
#             pred_ = predict_in_batches(model, comb_all_reshape, batch_size=1000)

# print("Processing predictions...")
# lc_pred_ = pred_.argmax(axis=1)

# # Reshape back to image dimensions
# img_pred = np.reshape(lc_pred_, (ds_ndvi.RasterYSize, ds_ndvi.RasterXSize))

# print(f"Prediction shape: {img_pred.shape}")
# print(f"Prediction min: {img_pred.min()}, max: {img_pred.max()}")
# print(f"Unique classes: {np.unique(img_pred)}")

# # Export result
# outFile = r'/home/jupyter-bryan/ISA_Data/Raster_02/ISA_MND_Multi_Orig_MCTNet.tif'
# raster.export(img_pred, ds_ndvi, filename=outFile, dtype='int')
# print(f"Prediction saved to: {outFile}")

In [21]:
%%time
print("\n=== Starting prediction with ALPE model ===")
try:
    pred_ = predict_in_batches_with_mask(
        model, 
        comb_all, 
        confidence_mask_final,  # ← Pass masks!
        batch_size=2000)
    
except Exception as e:
    print(f"Error during prediction: {e}")
    print("Trying with smaller batch size...")
    try:
        pred_ = predict_in_batches_with_mask(
            model, 
            comb_all, 
            confidence_mask_final,
            batch_size=1000)
        
    except Exception as e2:
        print(f"Still getting error: {e2}")
        print("Trying CPU-only prediction...")
        with tf.device('/CPU:0'):
            pred_ = predict_in_batches_with_mask(
                model, 
                comb_all, 
                confidence_mask_final,
                batch_size=500)

print("\n=== Processing predictions ===")
lc_pred_ = pred_.argmax(axis=1)

# Reshape back to image dimensions
img_pred = np.reshape(lc_pred_, (ds_ndvi.RasterYSize, ds_ndvi.RasterXSize))

print(f"Prediction shape: {img_pred.shape}")
print(f"Prediction min: {img_pred.min()}, max: {img_pred.max()}")
print(f"Unique classes: {np.unique(img_pred)}")
print(f"Class distribution:")
for cls in np.unique(img_pred):
    count = (img_pred == cls).sum()
    print(f"  Class {cls}: {count:,} pixels ({100*count/img_pred.size:.2f}%)")

# Export result
outFile = r'/home/jupyter-bryan/ISA_Data/Raster_02/ISA_DKI_Multi_Orig_withALPE.tif'
raster.export(img_pred, ds_ndvi, filename=outFile, dtype='int')
print(f"\n✓ Prediction saved to: {outFile}")


=== Starting prediction with ALPE model ===
Predicting 10,051,518 pixels in batches of 2,000...


2025-12-19 17:34:15.870718: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


Processed batch 10/5026
Processed batch 20/5026
Processed batch 30/5026
Processed batch 40/5026
Processed batch 50/5026
Processed batch 60/5026
Processed batch 70/5026
Processed batch 80/5026
Processed batch 90/5026
Processed batch 100/5026
Processed batch 110/5026
Processed batch 120/5026
Processed batch 130/5026
Processed batch 140/5026
Processed batch 150/5026
Processed batch 160/5026
Processed batch 170/5026
Processed batch 180/5026
Processed batch 190/5026
Processed batch 200/5026
Processed batch 210/5026
Processed batch 220/5026
Processed batch 230/5026
Processed batch 240/5026
Processed batch 250/5026
Processed batch 260/5026
Processed batch 270/5026
Processed batch 280/5026
Processed batch 290/5026
Processed batch 300/5026
Processed batch 310/5026
Processed batch 320/5026
Processed batch 330/5026
Processed batch 340/5026
Processed batch 350/5026
Processed batch 360/5026
Processed batch 370/5026
Processed batch 380/5026
Processed batch 390/5026
Processed batch 400/5026
Processed

ERROR 1: PROJ: proj_create_from_name: Open of /opt/tljh/user/share/proj failed
